In [1]:
# import libraries
from pyopenms import *
import os

In [2]:
# change directory to find file of interest
os.chdir(r'C:\Users\miar\Desktop\data')

In [3]:
#------------------------------------------------------------------------------------------------------------------------------

# ONLY NEED TO RUN THE NEXT TWO LINES ONCE

#-------------------------------------------------------------------------------------------------------------------------------

In [4]:
# load the content of the mzML file into the exp variable of type MSExperiment
#exp = MSExperiment()
#MzMLFile().load("HEK293T_De_Novo_053122_Glu-C_B_correctRTSenzyme_BP.mzML", exp)

# access the raw data and spectra
#spectrum_data = exp.getSpectrum(0).get_peaks()
#spectrum_data

In [5]:
# loop through the spectra to gather MS2 scans
#specM2 = []
#for s in exp.getSpectra():
#    if s.getMSLevel() == 2:
#        specM2.append(s)
        
#print("Number of MS2 scans: " + str(len(specM2)))

#exp.setSpectra(specM2) # keep only MS2

# store the modified data structure on disk
#MzMLFile().store("filtered_MS2.mzML", exp)

In [6]:
# parse function
def parseScanLine(input):
    x = input.split(" For: ")
    [scan_number, mzs] = x[1].split(", ")
    [precursor_mz, fragment_mz] = mzs.split(";")
    return [scan_number, precursor_mz, fragment_mz]

In [7]:
# checking lines of log file and creating dictionary of scan numbers and fragment mzs
try:
  
    # words to search for
    search = ' Submitted Custom Scan For:'
  
    # reading file content line by line
    scans = []
    frag = []
    search = ' Submitted Custom Scan For:'   # words to search for
    
    with open('App-2022-05-31_20-49-35.log') as f:
        for line in f:
            if search in line:
                scan_number, precursor_mz, fragment_mz = parseScanLine(line)
                scans.append(scan_number)
                frag.append(fragment_mz)
    
    # trim fragment strings to remove \n
    fragments = [sub[ : -1] for sub in frag]
    
    # convert lists to dictionary 
    # keys - scans 
    # values - fragments 
    dict1 = dict(zip(scans, fragments))
                  
    # if the input string doesn't exist in the text file
    if len(scans)==0 or len(frag)==0:
        print("\n\"" +search+ "\" is not found in \"" +'App-2022-05-31_20-49-35.log'+ "\"!")
    else:
        pass

except FileNotFoundError:
    print("The file does not exist!")

In [8]:
#dict1

In [9]:
# load in MS2 scans
exp1 = MSExperiment()
MzMLFile().load("filtered_MS2.mzML", exp1)

In [10]:
# convert scan numbers in dict to list
scan_nrs = list(dict1.keys())

# filter spectra for scan numbers also found in dict
filtered = MSExperiment()
spec_scans = []
spectra = []
for k, s in enumerate(exp1):
    if k in list(map(int, scan_nrs)): # convert str to int for every element in list
        filtered.addSpectrum(s)
        spec_scans.append(k)
        spectra.append(s)

# next, create peptide object ...
# https://pyopenms.readthedocs.io/en/latest/aasequences.html 

In [41]:
spec_scans

[1707,
 2140,
 2534,
 3032,
 3148,
 3582,
 3716,
 4026,
 4027,
 4056,
 4057,
 4067,
 4068,
 4072,
 4080,
 4111,
 4118,
 4130,
 4132,
 4140,
 4149,
 4150,
 4161,
 4171,
 4176,
 4195,
 4209,
 4214,
 4231,
 4239,
 4246,
 4272,
 4276,
 4281,
 4285,
 4289,
 4292,
 4297,
 4298,
 4299,
 4305,
 4323,
 4326,
 4327,
 4332,
 4349,
 4353,
 4354,
 4355,
 4365,
 4366,
 4370,
 4382,
 4383,
 4403,
 4405,
 4410,
 4429,
 4430,
 4443,
 4444,
 4452,
 4453,
 4458,
 4463,
 4476,
 4480,
 4503,
 4506,
 4507,
 4510,
 4531,
 4537,
 4539,
 4549,
 4562,
 4574,
 4580,
 4584,
 4592,
 4597,
 4599,
 4603,
 4605,
 4609,
 4614,
 4615,
 4618,
 4629,
 4635,
 4647,
 4649,
 4656,
 4659,
 4666,
 4673,
 4680,
 4685,
 4686,
 4689,
 4695,
 4701,
 4719,
 4721,
 4725,
 4727,
 4750,
 4774,
 4775,
 4776,
 4781,
 4787,
 4794,
 4801,
 4803,
 4806,
 4811,
 4818,
 4819,
 4822,
 4825,
 4827,
 4836,
 4842,
 4848,
 4856,
 4862,
 4867,
 4869,
 4873,
 4880,
 4882,
 4890,
 4896,
 4908,
 4912,
 4914,
 4915,
 4919,
 4927,
 4933,
 4937,
 4938,

In [11]:
# read in peptide sequence from tsv
import pandas as pd
tsv = pd.read_csv('HEK293T_De_Novo_053122_Glu-C_B_correctRTSenzyme_BP_realtimesearch1.tsv', sep='\t')

# create dictionary with scan # as key and sequence/charge as values
dict2 = dict([(i, [x,y]) for i, x,y, in zip(tsv['Scan Number'], tsv['Peptide'], tsv['Charge State'])])

In [42]:
seqs

['D.ASPDPMLEPM[15.9949]SWRGNIHE.F',
 'D.IFKNGSKRMIATVPLRHSIRD.R',
 'E.RQVPLASPSSM[15.9949]SAALRGISCYLKE.L',
 'D.FLCQASVGKSPGNNAADPFE.L',
 'E.KAQAIIATMGGTTLMASAHSWAKSSSFSCPE.-',
 'E.KSSM[15.9949]KRKLPFTISPSRNE.E',
 'E.AVCGAGPPSGLLPLPGPDPGPRGSQGSPTPTRAPASGPE.R',
 'E.SRPTPMGPPPTSHFHVLAD.T',
 'E.SIYVIRGTLFSVAVSFTD.Q',
 'D.GRGPPGNPPRRM[15.9949]GRINHLRGPSPPPMAGGUGR.-',
 '-.MASCAEPSEPSAPLPAGVPPLE.D',
 '-.MCHTSHSSGCPMACPGSPCCVPSTCYPPE.G',
 'E.KLM[15.9949]QLNLCSNRLE.S',
 'D.SVPPISQLPCTALSPVAQTHGSM[15.9949]LSPGAQPHD.S',
 'D.RFNHFSLTLNTNHGHILVD.Y',
 nan,
 'E.VVSVRNYGAAFHLPTSQRGE.I',
 nan,
 'D.SKVALSTCNGLHGM[15.9949]FE.D',
 'D.RRYCNSHLQVLGFIPKKE.R',
 'D.KPATDPVLQVVYKSQGKE.S',
 '-.MPRAQPSSASYQPVPADPFAIVSRAQQM[15.9949]VE.I',
 'D.SFWSCNVTFNISSKSILVD.K',
 'E.IQSYSSLTAAPFE.T',
 'E.GQAAARGAGPRLLLLLLLPLPLPLPLLQLVGRRPGAAGRSGSAWRRAQLGM[15.9949].-',
 'D.HAVVVGVYRPPPKVKN.-',
 'E.WGTRAKLHTVAICFQRGSE.K',
 'D.AYYRRVKRLYSNWRKGE.D',
 'E.CSRAGGNCCKKCTLTHD.A',
 'E.NLSKLKSLLTQLRAE.D',
 '-.SPSSIALFYKACLCSVDPVAM[15.99

In [12]:
#dict2

In [13]:
# isolate sequences from dict2 for scan numbers found in both dict2 and filtered spectra
seqs = []
for i in spec_scans:
    if i in dict2:
        seqs.append(dict2[i][0])
        
# isolate portion of sequence between the periods       
seqs_str = list(map(str, seqs)) # converts NaN (float) lines (and all lines) to str
trimmed_seqs = [] 
for i in seqs_str: 
    if i != 'nan':
        trimmed_seqs.append(i[2:-2]) # remove first two and last two characters in str 
    elif i == 'nan':
        pass # disregard NaN values

In [14]:
# if scan number in spectra data is also found in dict, create a peptide object 
pept_objects = []
for i in trimmed_seqs:
    pept_objects.append(AASequence.fromString(i))

In [15]:
# for the first peptide object/sequence in list...

# check which fragments are present in the spectrum
# loop through each prefix and suffix (b and y ions, respectively)
try:
    # y and b ions
    y_ions = []
    b_ions = []
    for i in range(1, (len(trimmed_seqs[0]) - 1)): # start at index of 1, end at peptide length - 1
        y_ions.append(pept_objects[0].getSuffix(i))
        b_ions.append(pept_objects[0].getPrefix(i))
        
except RuntimeError: # range above may be too large for indexing when considering modifications (ex. [15.9949])
    print('Modifications resulted in abnormal indexing for sequence: ' + str(pept_objects[0])) 
    
    # will have to remove "fragment" that is now the entire peptide length instead of peptide length - 1
    del y_ions[-1]
    del b_ions[-1]

Modifications resulted in abnormal indexing for sequence: ASPDPMLEPM[15.9949]SWRGNIHE


In [16]:
# for each ion look through all possible charge states

# isolate charges from dict2
char = []
for i in spec_scans:
    if i in dict2:
        char.append(dict2[i][1])

# remove charges in list accociated with a NaN sequence 
nan_indx = [x for x, z in enumerate(seqs_str) if z == 'nan'] # list of indexes containing NaN
charges = [i for j, i in enumerate(char) if j not in nan_indx] # list of charges after NaNs removed



In [17]:
# for the first sequence
c1 = charges[0]

if c1 == 2:
    for i in range(1, 2):
        pass
    
elif c1 == 3:
    # compute all y_ion mzs
    y_ion_mzs = []
    for i in y_ions:
        for y in range(1, 3):
            mz_y = i.getMonoWeight(Residue.ResidueType.YIon, y) / y
            y_ion_mzs.append(mz_y)
    # compute all b_ion mzs
    b_ion_mzs = []
    for i in b_ions:
        for y in range(1, 3):
            mz_b = i.getMonoWeight(Residue.ResidueType.YIon, y) / y
            b_ion_mzs.append(mz_b)
            
elif c1 == 4:
    for i in range(1, 4):
        pass
elif c1 == 5:
    for i in range(1, 5):
        pass
elif c1 == 6: #max(charges) = 6
    for i in range (1, 6):
        pass 
        
        

In [18]:
y_ion_mzs

[148.060435753871,
 74.53385611032101,
 285.119347977171,
 143.06331222197102,
 398.203412328071,
 199.60534439742102,
 512.246340519471,
 256.626808493121,
 569.2678046151709,
 285.137540540971,
 725.368915997971,
 363.188096232371,
 911.448229316971,
 456.227752891871,
 998.4802584764711,
 499.743767471621,
 1014.475158476471,
 507.741217471621,
 1111.5279226997711,
 556.267599583271,
 1240.5705169230712,
 620.788896694921,
 1353.654581273971,
 677.330928870371,
 1484.695066291071,
 742.851171378921,
 1581.7478305143711,
 791.377553490571,
 1696.774774673871,
 848.891025570321,
 1793.8275388971708,
 897.417407681971,
 1880.8595680566707,
 940.9334222617209]

In [19]:
b_ion_mzs

[90.054955690071,
 45.531116078421,
 177.08698484957102,
 89.047130658171,
 274.139749072871,
 137.573512769821,
 389.16669323237096,
 195.086984849571,
 486.2194574556709,
 243.61336696122098,
 617.2599424727708,
 309.13360946977093,
 730.3440068236708,
 365.67564164522094,
 859.3866010469708,
 430.19693875687096,
 956.4393652702707,
 478.72332086852094,
 972.4342652702708,
 486.72077086852096,
 1059.4662944297709,
 530.236785448271,
 1245.5456077487709,
 623.276442107771,
 1401.6467191315708,
 701.326997799171,
 1458.6681832272707,
 729.8377298470209,
 1572.7111114186707,
 786.8591939427209,
 1685.7951757695707,
 843.4012261181709,
 1822.8540879928707,
 911.930682229821]

In [20]:
help(MSSpectrum)

Help on class MSSpectrum in module pyopenms.pyopenms_5:

class MSSpectrum(builtins.object)
 |  Cython implementation of _MSSpectrum
 |  
 |  Documentation is available at http://www.openms.de/current_doxygen/html/classOpenMS_1_1MSSpectrum.html
 |   -- Inherits from ['SpectrumSettings', 'RangeManager1']
 |  
 |  The representation of a 1D spectrum.
 |  Raw data access is proved by `get_peaks` and `set_peaks`, which yields numpy arrays
 |  Iterations yields access to underlying peak objects but is slower
 |  Extra data arrays can be accessed through getFloatDataArrays / getIntegerDataArrays / getStringDataArrays
 |  See help(SpectrumSettings) for information about meta-information
 |  -----
 |  Usage:
 |  ms_level = spectrum.getMSLevel()
 |  rt = spectrum.getRT()
 |  mz, intensities = spectrum.get_peaks()
 |  
 |  Methods defined here:
 |  
 |  __copy__(...)
 |  
 |  __deepcopy__(...)
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Ret

In [21]:

s.findNearest(b_ion_mzs[8], 0.4)

-1

In [40]:
spectra[0].findNearest(y_ion_mzs[6] , 0.4) 

-1

In [32]:
for s in spectra: # BAD, fix in AM
    # only check the scan the precursor was originally associated with from dict2
    output = s.findNearest(y_ion_mzs[1], 0.4)
    if output != -1:
        print(output)

0
0
0
2
0
4
4
4
3
4
5
2
1
5


In [27]:
len(spectra)

3457